## Define query

In [38]:
from ipyleaflet import Map, Marker, Circle
from ipywidgets import widgets, HBox, VBox

center = (38.92, -77.05)  # Example: New York City
m = Map(center=center, zoom=10)
marker = None
circle = None
current_center = center
current_radius = 1
def handle_click(**kwargs):
    global marker, circle, current_center
    if kwargs.get('type') == 'click':
        latlng = kwargs.get('coordinates')
        current_center = latlng  # Update the center
        if marker is not None:
            m.remove_layer(marker)
        if circle is not None:
            m.remove_layer(circle)
        marker = Marker(location=latlng)
        m.add_layer(marker)
        update_circle(latlng, radius_slider.value)



def query_geodata(center, radius):
    import geopandas as gpd
    # This is a placeholder function. You would replace it with your actual query logic.
    # For demonstration, it returns an empty GeoDataFrame.
    return gpd.GeoDataFrame({'name': [], 'geometry': []})



def update_circle(center, radius):
    global circle, current_radius
    current_radius = radius  # Update the radius
    if circle is not None:
        m.remove_layer(circle)
    circle = Circle()
    circle.location = center
    circle.radius = radius
    circle.color = "red"
    circle.fill_color = "red"
    m.add_layer(circle)
m.on_interaction(handle_click)


radius_slider = widgets.IntSlider(
    value=100,    # Starting value in meters
    min=100,      # Minimum value in meters
    max=5000,     # Maximum value in meters (5 km)
    step=100,     # Step size in meters
    description='Radius (m):',
    continuous_update=False
)

def on_radius_change(change):
    global marker
    if marker is not None:
        update_circle(marker.location, change['new'])

radius_slider.observe(on_radius_change, names='value')


display(VBox([m, radius_slider]))


In [32]:
#current_center, current_radius = [38.95056502857071, -76.90616067903447], 2000

In [39]:
print(current_center, current_radius)

[38.90428983816469, -77.03413819449803] 600


## Spatial query

In [40]:
import pygeohash as pgh
import geopandas as gpd
import pandas as pd
import sys
import os
import time
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from geohashtree.geohash_func import geohashes_covering_circle, bounding_box,geohash_to_gdf
from geohashtree.trie import trim_hashes
from geohashtree.geohashtree import LiteTreeOffset,FullTreeFile
from geohashtree.filesystem import ipfs_get_index_folder,extract_and_concatenate_from_ipfs
from shapely.geometry import Point

precision = 4
result_hashes = geohashes_covering_circle(current_center[1],current_center[0],current_radius/1000,precision,distance_type='haversine')

local_index_path = "../data/test/osm_us_poi_gh_sorted"
index_cid = "bafybeib67cdvhxou7rbvgmxwbwdr3ozygwv2urjfvh6p3xbh4b3jhjmal4"


gdf_rand_points = gpd.GeoDataFrame(geometry=[Point(current_center[1], current_center[0])], crs='EPSG:4326')  # Start with WGS84
gdf_rand_points = gdf_rand_points.to_crs(epsg=3857)

mode = 'offline'
tree = LiteTreeOffset(mode=mode)
t0 = time.time()
if tree.mode == 'offline':
    if not os.path.exists(local_index_path):
        print('caching to',local_index_path)
        ipfs_get_index_folder(index_cid,local_index_path)
else:
    local_index_path = index_cid
t1 = time.time()
#tree.count(result_hashes,local_index_path)
t2 = time.time()
retr = tree.retrieve(result_hashes,local_index_path)
gdf_radius = gpd.GeoDataFrame({'geometry':gdf_rand_points.buffer(current_radius)},crs='EPSG:3857').to_crs('EPSG:4326')
result = gpd.sjoin(retr,gdf_radius).query('amenity=="restaurant"')
t3 = time.time()

print(f'query finished with feature number: {result.shape[0]}')
print(f'index caching: {t1-t0:.2f}s')
#print(f'fuzzy count: {t2-t1:.2f}s')
print(f'exact query: {t3-t2:.2f}s')

Index Mode: offline
12908
0 131
comb 1
0.007905006408691406 23.13961911201477
query finished with feature number: 46
index caching: 0.00s
exact query: 23.22s


In [41]:
# dc_outline = gpd.read_file("../data/maryland_demo/Washington_DC_Boundary.geojson")
# df_gh = pd.concat([geohash_to_gdf(geohash) for geohash in result_hashes]).to_crs(epsg=3857)
# ax = df_gh.plot(figsize=(10, 10), alpha=0.3, edgecolor="k")
# dc_outline.to_crs(epsg=3857).plot(ax=ax, facecolor='none',edgecolor="k")
# gdf_radius.to_crs(epsg=3857).plot(ax=ax,alpha=0.2,color='red')
# result.query('amenity=="restaurant"').to_crs(epsg=3857).plot(ax=ax, color='green', markersize=7, label='Restaurants')

## Display result

In [42]:
import folium
import geopandas as gpd

gdf = result
# Create a Folium map centered around the mean location of the points
# Create a Folium map centered around the mean location of the points
m = folium.Map(location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], zoom_start=13)

# Add points from the GeoDataFrame to the map with popups
for _, row in gdf.iterrows():
    popup_content = f"{row['name']}<br><a href='{row['website']}' target='_blank'>{row['website']}</a>"
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup_content,
    ).add_to(m)

# Define the latitude, longitude, and radius for the circle
circle_lat,circle_lng = current_center  # Example latitude  # Example longitude
circle_radius = current_radius  # Radius in meters

# Add a circle to the map
folium.Circle(
    location=[circle_lat, circle_lng],
    radius=circle_radius,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.3,
    popup='Circle Area',
).add_to(m)

# Display the map
m